<a href="https://colab.research.google.com/github/EP111111/Doc_analysis/blob/main/DA_test_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()


In [ ]:
!pip install shapely==1.8.5
!pip install pypdf
!pip install google-cloud-aiplatform --upgrade
!pip install python-docx
!pip install langchain
!pip install docx2txt
!pip install chromadb
!pip install langchainhub

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel
from langchain.llms import VertexAI
from google.colab import drive
import docx
import numpy as np
import pandas as pd

In [ ]:
import time
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
import pandas as pd
import json
from functions import *

In [ ]:
PROJECT_ID = "ics-analysis-dev"
vertexai.init(project=PROJECT_ID, location="us-central1")

model = TextGenerationModel.from_pretrained("text-bison@001")

##Document Splitting
Document Splitting is to split documents into smaller chunks.

In [ ]:
# from langchain.document_loaders import Docx2txtLoader
# text=Docx2txtLoader("Project Alpha.docx").load()

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("Protected Collaboration Application - RFI Annex.pdf")

#Load the document by calling loader.load()
text = loader.load()


In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(text)


##Vector Stores and Embeddings
 put these chunks into an index so that we are able to retrieve them easily when we want to answer questions on this document.

In [ ]:
embeddings = VertexAIEmbeddings()
# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="/content"

)

print(vectordb._collection.count())

11


In [ ]:
from langchain.chat_models import ChatVertexAI
llm = ChatVertexAI()

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
question = """extract all questions in this document  :
"""
result = qa_chain({"query": question})
print(result["result"])

import re

questions = re.split(r'\d+\.', result["result"])
questions = [q.strip() for q in questions if q.strip()]

# for i, question in enumerate(questions, start=1):
#     print(f"{i}. {question}")

 1. Does the solution allow users to securely delete messages from their endpoints? How are messages deleted from devices and servers? 
2. How is data treated, stored, and deleted both inside the app and on the backend servers? How can the solution support sending some messages, including decisions, to an official* record keeping system for filing ? How can the solution allow users to toggle the classification of a message, attachment or meeting? 
3. Is the solution or its cryptography open source? 
4. Can the solution support screen sharing? 
5. How does the solution support multi -format file sharing? Please


## Test 1

In [ ]:
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
retriever=vectordb.as_retriever()
rag_prompt = hub.pull("rlm/rag-prompt")
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

In [ ]:
result = rag_chain.invoke("""extract each question one by one from this document :
""")

questions = re.split(r'\d+\.', result.content)
questions = [q.strip() for q in questions if q.strip()]

for i, question in enumerate(questions, start=1):
    print(f"{i}. {question}")

1. Does the solution allow users to securely delete messages from their endpoints? How are messages deleted from devices and servers?
2. How is data treated, stored, and deleted both inside the app and on the backend servers?
3. Is the solution or its cryptography open source?


## Test 2

In [ ]:

from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "question": {"type": "string"},
    },
    "required": ["name"],
}
llm = ChatVertexAI(temperature=0, model="text-bison@001")
chain = create_extraction_chain(schema, llm)
chain.run(docs[1].page_content)

# for i in range (len(docs)):
#   chain = create_extraction_chain(schema, llm)
#   chain.run(docs[i].page_content)

TypeError: ignored

In [ ]:
print(docs[1].page_content)
len(docs)

PROTECTED level. This enterprise level capability will operate across multiple disassociated 
infrastructures. The solution should provide seamless, easy to use, collaboration between users and 
offer premium tec hnical support.  
The solution must meet PROTECTED security requirements and demonstrate, with technical detail, its ability to maintain this security standard, including whilst travelling abroad.  
3.2 User Support  
The solution is for consumption by senior decision makers and as such it is intended that ‘white glove 
service’ will be detailed as part of the RF I. This includes 24/7 Australian security cleared technical 
support personnel available for on- site support.   
3.3 Case studies  
Please provide, if available, relevant case studies of your solution by other Australian Government 
users and a point of contact if available.  
Note to tenderers:  The following s ections address the specific information that is sought from


11

## Test 3

In [ ]:
from traitlets import Int
from typing import Sequence, Optional
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.llms import OpenAI
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser



In [ ]:
class question(BaseModel):
    question_content: str

class questions(BaseModel):
    """Identifying information about all questions in a text."""
    questions: Sequence[question]



# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=questions)

# Prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Run
_input = prompt.format_prompt(query=docs[1].page_content)

llm_output = llm(_input.text)

parser.parse(llm_output)

questions(questions=[question(question_content='What is the security level of the solution?'), question(question_content='What is the user support for the solution?'), question(question_content='Are there any case studies of the solution being used by other Australian Government users?')])

## Text Extraction


In [ ]:

# def take_context(question,cont):
#   ques=f'''
#   Document:{cont}
#   ___above this line is the document,below this line is what you need to do___
#   EXTRACT context related to the   question:["{question}"]. The output must be analysed from the above Document,

#   if no context related to the input exist, return:"no" only

#   question:["{question}"]

#   output:context OR "No"
#   '''

#   result = qa_chain({"query": question})


#   return result["result"]

def take_context(question,cont,model):
    context=empty_Vertex(f'''
    Document:{cont}
    ___above this line is the document,below this line is what you need to do___
    EXTRACT context related to the question. The output must be from the above Document, if no context related to the input exist, return:"no" only

    question:["{question}"]
    output:context OR "no"
    ''',model)
    return context




In [ ]:
def docs_context(ques,docs):
  print("question: " + ques +"\n\nThere are "+str(len(docs))+ " sections\n")


  for i in range(len(docs)):
    res= take_context(ques,docs[i].page_content,model)
    print("\nsection "+str(i)+":\ncontext related to the question :"+ res)
    print("##################")
docs_context(questions[4],docs)

question: How does the solution support multi -format file sharing? Please

There are 11 sections


section 0:
context related to the question :no
##################

section 1:
context related to the question :no
##################

section 2:
context related to the question :no
##################

section 3:
context related to the question :no
##################

section 4:
context related to the question :no
##################

section 5:
context related to the question :no
##################

section 6:
context related to the question :no
##################

section 7:
context related to the question :How does the solution support multi -format file sharing? Please provide a list of the file types 
supported.
##################

section 8:
context related to the question :no
##################

section 9:
context related to the question :no
##################

section 10:
context related to the question :no
##################
